In [3]:
open_ai_key = open('open_ai_key.txt', 'r').read()
os.environ['OPENAI_API_KEY'] = open_ai_key
google_key = open('google_key.txt', 'r').read()
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = google_key

In [1]:
import os
from openai import OpenAI
import re
import json
from IPython.display import clear_output
from IPython.display import display, Image
import base64
from io import BytesIO
from PIL import Image as PilImage
import io
from google.cloud import vision
import time as timer

In [5]:
def detect_food_items(img):
    """Combines label and text detection for better accuracy on food items."""
    client = vision.ImageAnnotatorClient()
    image = vision.Image(content=img)

    # Perform label detection
    label_response = client.label_detection(image=image)
    labels = label_response.label_annotations

    # Perform text detection
    text_response = client.text_detection(image=image)
    texts = text_response.text_annotations

    return labels, texts

In [7]:
def detect(img_path, crop_limit=3):
    # Opens a image in RGB mode
    im = PilImage.open(img_path)
    width, height = im.size
    labels = set()
    texts = set()

    # Now crop over the picture and check again with multiple crop sizes
    crops = 1
    while crops <= crop_limit:
        for i in range(crops):
            top = max((i/crops-1/(crops*8)),0)*height
            bottom = min((9/(crops*8)+i/crops),1)*height
            for j in range(crops):
                left = max((j/crops-1/(crops*8)),0)*width
                right = min((9/(crops*8)+j/crops),1)*width
                im_crop = im.crop((left, top, right, bottom))
                b = io.BytesIO()
                im_crop.save(b, 'jpeg')
                im_bytes = b.getvalue()
                l, t = detect_food_items(im_bytes)
                for label in l:
                    labels.add(label.description)
                for text in t:
                    texts.add(text.description)
        crops+=1
    return labels, texts

In [9]:
def scan_pantry(client, img_path, crop):
    labels, texts = detect(img_path, crop_limit=crop)
    foods = client.chat.completions.create(
      model="gpt-4o",
      response_format={"type": "json_object"},
      messages=[
        {"role": "system", "content": "You are a list reader, skilled at reading multiple lists and synthesizing them, then outputting that list in JSON."},
        {"role": "user", "content": f"This is a list of objects that were labeled in an image of a pantry (containing both food items and other objects): {labels}\nMany things are labeled but some are not. This is a list of text that was recognized in the image (labels on boxes, nutrition info, etc.): {texts}\nUsing information from both lists and knowing that they came from reading/analyzing the same image, create a list of the food items that are in this pantry. The JSON returned should have a single key, 'food items', with its value being the created list."}
      ]
    )
    return foods.choices[0].message.content

In [27]:
def ai_get_recipe_json(client, dish, experience, servings, allergies, pantry):
    """Fetches a recipe by name and experience level using AI and returns JSON."""
    if pantry:
        recipe = client.chat.completions.create(
          model="gpt-4o",
          response_format={"type": "json_object"},
          messages=[
            {"role": "system", "content": "You are a chef, skilled at creating and finding recipes for anyone and anything designed to output JSON."},
            {"role": "user", "content": f"Create a recipe for {dish} for a person of {experience} experience. Include amounts of each ingredient for {servings} and avoid {allergies} due to allergies. It is known that the person has these {pantry} (there may be a few things that were not scanned, but expect that most available ingredients were scanned). Be sure to inlcude the name of the dish and the total time."}
          ]
        ).choices[0].message.content
    else:
        recipe = client.chat.completions.create(
          model="gpt-4o",
          response_format={"type": "json_object"},
          messages=[
            {"role": "system", "content": "You are a chef, skilled at creating and finding recipes for anyone and anything designed to output JSON."},
            {"role": "user", "content": f"Create a recipe for {dish} for a person of {experience} experience. Include amounts of each ingredient for {servings} and avoid {allergies} due to allergies. Be sure to inlcude the name of the dish and the total time."}
          ]
        ).choices[0].message.content
    return recipe

def convert_to_json(client, recipe):
    """Converts existing recipe to JSON using AI."""
    json_recipe = client.chat.completions.create(
      model="gpt-4o",
      response_format={"type": "json_object"},
      messages=[
        {"role": "system", "content": "You are a recipe reader, skilled at reading recipes and converting them to JSON."},
        {"role": "user", "content": recipe},
        {"role": "user", "content": "Convert that recipe to JSON. Include recipe name, total time, servings, and amounts for each ingredient"}
      ]
    ).choices[0].message.content
    return json_recipe

def adjust_servings(client, recipe, servings):
    """Modifies the recipe for the new servings."""
    # adjust recipe
    new_recipe = client.chat.completions.create(
      model="gpt-4o",
      response_format={"type": "json_object"},
      messages=[
        {"role": "system", "content": "You are an recipe reader, skilled at adjusting ingredient amounts for a different number of servings, and designed to output in JSON"},
        {"role": "assistant", "content": recipe},
        {"role": "user", "content": f"I need to make that exact recipe, but for {servings} servings. Modify the servings listed and the ingredient amounts accordingly."}
      ]
    ).choices[0].message.content
    return new_recipe

def modify_for_pantry(client, recipe, pantry):
    """Modifies the recipe to exclude missing ingredients using AI."""
    new_recipe = client.chat.completions.create(
      model="gpt-4o",
      response_format={"type": "json_object"},
      messages=[
        {"role": "system", "content": "You are an accomodating chef, skilled at modifying recipes so they don't require certain ingredients, and designed to output the modified recipe in JSON."},
        {"role": "assistant", "content": recipe},
        {"role": "user", "content": f"I like this recipe: {recipe} but I may not have all of the ingredients. I know I have these {pantry} (an I may have a few others as well). Modify the recipe so it is similar but modified accordingly."}
      ]
    ).choices[0].message.content
    return new_recipe

def create_grocery_list_ai(client, ingredients, have):
    """Creates a grocery list for ingredients that user does not have."""
    # load ingredient list
    ingredients = json.loads(client.chat.completions.create(
      model="gpt-4o",
      response_format={"type": "json_object"},
      messages=[
        {"role": "system", "content": "You are an ingredient list reader, skilled at reading ingredient lists and converting them to JSON with ingredient-amount pairs."},
        {"role": "assistant", "content": ingredients},
        {"role": "user", "content": "Convert that ingredient list to JSON with ingredient-value pairs"}
      ]
    ).choices[0].message.content)

    # check which ingredients user has
    yes = ('y','Y','yes','Yes','YES')
    no = ('n','N','no','No','NO')
    for i in ingredients:
        if i in have:
            continue
        while True:
            check = input(f"Do you have {ingredients[i]} {i} (y/n)? ")
            if check in yes:
                have[i] = ingredients[i]
                break
            elif check in no:
                break
            print("Please enter 'y' or 'n'")

    # generate grocery list
    print("Generating grocery list...")
    list = client.chat.completions.create(
      model="gpt-4o",
      messages=[
        {"role": "system", "content": "You are a grocery list reader, skilled at reading multiple lists and synthesizing them to make a new one."},
        {"role": "assistant", "content": str(ingredients)},
        {"role": "user", "content": f"I have these ingredients: {str(have)}. I need these ingredients: {str(ingredients)}. Make me a grocery list of what I need but don't have, including the amount I should buy of each (1 14 oz can, 2 1lb packages, etc.)."}
      ]
    ).choices[0].message.content
    return list, have

def modify_recipe_ai(client, ingredients, recipe, have, missing):
    """Modifies the recipe to exclude missing ingredients using AI."""
    # load ingredient list
    ingredients = json.loads(client.chat.completions.create(
      model="gpt-4o",
      response_format={"type": "json_object"},
      messages=[
        {"role": "system", "content": "You are an ingredient list reader, skilled at reading ingredient lists and converting them to JSON with ingredient-amount pairs."},
        {"role": "assistant", "content": ingredients},
        {"role": "user", "content": "Convert that ingredient list to JSON with ingredient-value pairs"}
      ]
    ).choices[0].message.content)

    # check which ingredients user is missing
    yes = ('y','Y','yes','Yes','YES')
    no = ('n','N','no','No','NO')
    for i in ingredients:
        if i in have or i in missing:
            continue
        while True:
            check = input(f"Do you have {ingredients[i]} {i} (y/n)? ")
            if check in yes:
                have[i] = ingredients[i]
                break
            elif check in no:
                missing[i] = ingredients[i]
                break
            print("Please enter 'y' or 'n'")

    # modify recipe
    print("Generating modified recipe..", end="")
    new_recipe = client.chat.completions.create(
      model="gpt-4o",
      response_format={"type": "json_object"},
      messages=[
        {"role": "system", "content": "You are an accomodating chef, skilled at modifying recipes so they don't require certain ingredients, and designed to output the modified recipe in JSON."},
        {"role": "assistant", "content": recipe},
        {"role": "user", "content": f"I like this recipe: {recipe} but I don't have these ingredients: {str(missing)}. Modify the recipe so it is similar but avoids or replaces the ingredients I don't have."}
      ]
    ).choices[0].message.content
    return new_recipe, have, missing

def new_recipe_ai(client, dish, experience, servings, allergies, recipe, missing):
    """Fetches a new recipe based on dislikes of old recipe by name and experience level using AI and returns JSON."""
    dislikes = input("What do you dislike about the current recipe? ")
    print("Generating new recipe..", end="")
    new_recipe = client.chat.completions.create(
      model="gpt-4o",
      response_format={"type": "json_object"},
      messages=[
        {"role": "system", "content": "You are a chef, skilled at creating and finding recipes for anyone and anything designed to output JSON."},
        {"role": "assistant", "content": recipe},
        {"role": "user", "content": f"Create a recipe for {dish} for a person of {experience} experience, knowing that these were their dislikes of the previous recipe: {dislikes}. Include amounts of each ingredient for {servings} and avoid {allergies} due to allergies. Be sure to inlcude the name of the dish and the total time."}
      ]
    ).choices[0].message.content
    return new_recipe

def create_image_ai(client, name, ingredients):
    image = client.images.generate(
      model="dall-e-3",
      prompt=f"Create an image of a food called '{name}'. It uses the following ingredients: {ingredients}. The image must be of the actual finished food, not of the ingredients.",
      style="natural",
      size="1024x1024",
      quality="standard",
      response_format="b64_json",
      n=1,
    )
    return image.data[0].b64_json

def create_image_ai_steps(client, name, steps):
    image = client.images.generate(
      model="dall-e-3",
      prompt=f"Create an image of a food called '{name}'. It is made with the following steps: {steps}. The image must be of the FINISHED food, as it is SERVED, not of the process. NO TEXT OR RAW INGREDIENTS IN THE IMAGE.",
      style="natural",
      size="1024x1024",
      quality="standard",
      response_format="b64_json",
      n=1,
    )
    return image.data[0].b64_json

def show_image(image):    
    # Remove the Base64 header if present
    image = image.split(",")[-1]
    
    # Decode the Base64 string
    image_data = base64.b64decode(image)
    
    # Create an in-memory binary stream
    image_stream = BytesIO(image_data)
    
    # Display the image
    display(Image(data=image_stream.read(), width=300, height=300))

def print_recipe_ai(client, recipe, servings):
    """Prints the full recipe using AI."""
    # get name of recipe
    print(".", end="")
    name = client.chat.completions.create(
      model="gpt-4o",
      messages=[
        {"role": "system", "content": "You are a JSON reader, skilled at reading JSON and pulling out pieces you are asked for, whether the keys have the exact same name or not."},
        {"role": "assistant", "content": recipe},
        {"role": "user", "content": "Give me only the name of that recipe. Remove all quotation marks and replace underscores with a space. Do not label it (eg. with 'recipe name:'."}
      ]
    ).choices[0].message.content

    # get cook time of recipe
    print(".", end="")
    time = client.chat.completions.create(
      model="gpt-4o",
      messages=[
        {"role": "system", "content": "You are a JSON reader, skilled at reading JSON and pulling out pieces you are asked for, whether the keys have the exact same name or not."},
        {"role": "assistant", "content": recipe},
        {"role": "user", "content": "Give me only the time required of that recipe, in the form '{number} minutes'. Remove all quotation marks and replace underscores with a space."}
      ]
    ).choices[0].message.content

    # get ingredient list
    print(".", end="")
    ingredients = client.chat.completions.create(
      model="gpt-4o",
      messages=[
        {"role": "system", "content": "You are a JSON reader, skilled at reading JSON and pulling out pieces you are asked for, whether the keys have the exact same name or not."},
        {"role": "assistant", "content": recipe},
        {"role": "user", "content": "Give me only the ingredients/amounts of that recipe, with each ingredient on its own line, bulleted in the format 'ingredient, amount'. Remove all quotation marks and replace underscores with a space."}
      ]
    ).choices[0].message.content

    # get directions
    print(".", end="")
    steps = client.chat.completions.create(
      model="gpt-4o",
      messages=[
        {"role": "system", "content": "You are a JSON reader, skilled at reading JSON and pulling out pieces you are asked for, whether the keys have the exact same name or not."},
        {"role": "assistant", "content": recipe},
        {"role": "user", "content": "Give me only the steps of that recipe, numbered, with each step on its own line. Remove all quotation marks and replace underscores with a space."}
      ]
    ).choices[0].message.content

    # format and print parts as full recipe
    print(".", end="")
    image = create_image_ai_steps(client, name, steps)
    print(".", end="")
    print("\nHere's a suggested recipe based on your input:")
    show_image(image)
    print(f"\n{name}\nTotal time: {time}\nServings: {servings}\n\nIngredients:\n{ingredients}\n\nDirections:\n{steps}")
    return name, time, ingredients, steps

def to_file(name, time, servings, ingredients, steps):
    """Prints recipe to file"""
    # save printed recipe text
    text = f"\n{name}\nTotal time: {time}\nServings: {servings}\n\nIngredients:\n{ingredients}\n\nDirections:\n{steps}"
    while True:
        file_name = input("Enter file name to save the file to (must be a .txt; if it does not exist, the file will be created; if it does, it will be overwritten; make sure the file is not open): ")
        try: # ensure file can open
            f = open(file_name, "w")
        except:
            print("An error ocurred when opening this file. Please try again.")
            continue
        else:
            pass
            
        try: # ensure file can write
            f.write(text)
            f.close()
        except:
            print("An error ocurred when writing to this file. Please try again.")
        else:
            break
    return file_name

def clear():
    """Displays the first few lines after clearing output"""
    clear_output()
    print("Welcome to the AI-powered Cooking Assistant!\n")

def walkthrough_recipe(name, ingredients, steps):
    """Walks the user through the recipe step by step."""
    print(f"Let's start cooking {name}!")
    print("First, get your ingredients together:")
    print(ingredients)
    steps = steps.split("\n")
    input("Press enter once you have all of your ingredients together.")
    print("\nNow let's cook!\nPress enter after completing each step to move on to the next.")
    for step in steps:
        print("\n")
        input(step)
    print("\nRecipe complete! Enjoy your meal!\n")

In [33]:
# Main function to run the cooking assistant
def cooking_assistant():
    client = OpenAI()
    missing = {}
    have = {}
    
    print("Welcome to the AI-powered Cooking Assistant!")

    while True:
        # User options
        print("\nDo you want to upload picture(s) of your pantry/fridge to scan for ingredients?")
        print("1. Yes - I will not be going to the store and need a recipe that uses my ingredients.")
        print("2. No - I will be going to the store anyway.")
        
        choice = input("Enter your choice (1/2): ").strip()
        
        if choice == "1": # upload ingredient images
            print("\nLet's scan your pantry.")
            # Ask for user details
            speed = None
            while speed not in ('0','1'):
                speed = input("Enter '0' for fast or '1' for more accurate reading of your images: ")
            if speed == '0':
                speed = 3
            elif speed == '1':
                speed = 5
            elapses = []
            go = True
            while go:
                go = False
                pantry = set()
                image_paths = input("Enter the path of each image of your pantry/fridge (comma separated): ").split(',')
                print('Reading and analyzing images..', end='')
                for file in image_paths:
                    start = timer.time()
                    print('.', end='')
                    file = file.strip()
                    try:
                        foods = scan_pantry(client, file, crop=speed)
                    except:
                        print(f"{file} did not work. Make sure that you spelled them correctly and used the full path for each.")
                        go = True
                        break
                    else:
                        foods = json.loads(foods)
                        for key in foods.keys():
                            t = type(foods[key])
                            if t == type('1'):
                                pantry.add(foods[key])
                            elif t == type([1,2]) or t == type((1,2)):
                                for value in foods[key]:
                                    pantry.add(value)
                            elif t == type({1:2}):
                                for k in foods[key].keys():
                                    t = type(foods[key][k])
                                    if t == type('1'):
                                        pantry.add(foods[key][k])
                                    elif t == type([1,2]) or t == type((1,2)):
                                        for value in foods[key][k]:
                                            pantry.add(value)
                    end = timer.time()
                    elapses.append(end-start)
            with open('ingredients.txt', 'w') as f:
                f.write(str(pantry))
            print(f"The average time per image was: {sum(elapses)/len(elapses)}")
            break
    
        elif choice == "2": # upload and modify a recipe
            print("\nI'll keep your recipe open to any ingredients.")
            pantry = None
            break
    
        else: # something other than 1 or 2 was entered - tell user and ask again
            print("Invalid choice. Please enter only the number.")

    while True:
        # User options
        print("\nWhat would you like to do?")
        print("1. Create a recipe.")
        print("2. Upload and modify my own recipe.")
        
        choice = input("Enter your choice (1/2): ").strip()
        
        if choice == "1": # create a recipe
            print("\nLet's create a recipe.")
            # Ask for user details
            experience_level = input("What's your cooking experience level? ")
            allergies = input("Do you have any allergies? (comma separated): ")
            while True:
                try:
                    servings = int(input("How many servings are you cooking for? (number): ").strip())
                except:
                    print("That is not a number.")
                else:
                    break
            
            # Ask for desired dish
            dish_name = input("What do you want to cook? (e.g., chocolate cupcakes, pasta, pastries, dinner): ")
            print("\nGenerating recipe..", end="")
            recipe = ai_get_recipe_json(client, dish_name, experience_level, servings, allergies, pantry)
            break
    
        elif choice == "2": # upload and modify a recipe
            print("\nLet's modify your recipe.")
            file_name = input("Ensure your recipe is in a .txt file. Enter the file path here: ")
            while True:
                try: # ensure file can open
                    f = open(file_name, "r")
                except:
                    print("An error ocurred when opening this file. Please try again.")
                    continue
                else:
                    pass
                    
                try: # ensure file can write
                    recipe = f.read()
                except:
                    print("An error ocurred when reading this file. Please try again.")
                else:
                    break
            print("File read successfully.")
            servings = input("How many servings are you cooking for (same or different from what the original recipe makes)? ")
            print("Adjusting recipe..", end="")
            recipe = convert_to_json(client, recipe)
            recipe = adjust_servings(client, recipe, servings)
            if pantry:
                recipe = modify_for_pantry(client, recipe, pantry)
            dish_name = None
            break
    
        else: # something other than 1 or 2 was entered - tell user and ask again
            print("Invalid choice. Please enter only the number.")

    # Print recipe
    name, time, ingredients, steps = print_recipe_ai(client, recipe, servings)
    if dish_name is None:
        dish_name = name

    # Modify recipe until user likes it
    while True:
        # User options
        print("\nWhat would you like to do next?")
        print("1. I have all the ingredients - Start cooking!")
        print("2. Print this recipe.")
        print("3. I need to buy some ingredients - Create a grocery list.")
        print("4. I don't have some ingredients and can't buy them - Modify the recipe.")
        print("5. I don't like this recipe - Let's modify or make a new one.")
    
        choice = input("Enter your choice (1/2/3/4/5): ").strip()
    
        if choice == "1": # start cooking
            break

        elif choice == "2": # print the recipe
            print("\nLet's print your recipe.")
            file_name = to_file(name, time, servings, ingredients, steps)
            print(f"Your recipe has been saved to {file_name}")
    
        elif choice == "3": # create a grocery list
            print("\nLet's create your grocery list.")
            list, have = create_grocery_list_ai(client, ingredients, have)
            clear()
            print(list)
            input("Press enter once you have purchased all of your ingredients")
    
        elif choice == "4": # modify recipe
            print("\nLet's modify the recipe.")
            recipe, have, missing = modify_recipe_ai(client, ingredients, recipe, have, missing)
            clear()
            print("Generating modified recipe..", end="")
            name, time, ingredients, steps = print_recipe_ai(client, recipe, servings)
            input("Press enter once you have reviewed the new recipe. If you are still missing ingredients, you can modify again.")
    
        elif choice == "5": # new recipe
            print("\nLet's find another recipe.")
            recipe = new_recipe_ai(client, dish_name, experience_level, servings, allergies, recipe, missing)
            clear()
            print("Generating new recipe..", end="")
            name, time, ingredients, steps = print_recipe_ai(client, recipe, servings)
            input("Press enter once you have reviewed the new recipe. If you are still want to make changes, you can modify again.")
    
        else: # something other than 1, 2, 3, 4, or 5 was entered - tell user and ask again
            print("Invalid choice. Please enter only the number.")

    clear()
    walkthrough_recipe(name, ingredients, steps)

    # print recipe if necessary
    yes = ('y','Y','yes','Yes','YES')
    no = ('n','N','no','No','NO')
    while True:
        answer = input("In case you didn't print your recipe before, would you like to now (y/n)? ")
        if answer in yes:
            file_name = to_file(name, time, servings, ingredients, steps)
            print(f"Your recipe has been saved to {file_name}")
            break
        elif answer in no:
            break

    # sign off
    print("\nNow that you're done cooking, I'm done for now! Come back next time you need help from the AI-powered Cooking Assistant!")

In [35]:
#### Run the cooking assistant
if __name__ == "__main__":
    cooking_assistant()

Welcome to the AI-powered Cooking Assistant!

Let's start cooking Creamy Chicken and Vegetable Pasta!
First, get your ingredients together:
- ELBOWS Gluten-Free Pasta, 1 pound
- Chicken breasts, 1.5 pounds
- Frozen peas, 1 cup
- Frozen corn, 1 cup
- Yellow onion, 1 medium
- Garlic, 3 cloves
- Cooking oil, 2 tablespoons
- Cheddar cheese, 2 cups
- Milk, 2 cups
- Salt, 1 teaspoon
- Black pepper, 1/2 teaspoon
- Parsley, 2 tablespoons


Press enter once you have all of your ingredients together. 



Now let's cook!
Press enter after completing each step to move on to the next.




1. Cook the ELBOWS Gluten-Free Pasta according to the package directions. Drain and set aside. 


2. Heat the cooking oil in a large skillet over medium heat. Add the chopped onion and cook for 5 minutes, or until translucent. 


...

7. Add the cooked pasta to the skillet and toss to coat evenly with the creamy sauce. 


8. Remove from heat and garnish with chopped parsley before serving. 



Recipe complete! Enjoy your meal!



In case you didn't print your recipe before, would you like to now (y/n)?  
In case you didn't print your recipe before, would you like to now (y/n)?  
In case you didn't print your recipe before, would you like to now (y/n)?  n



Now that you're done cooking, I'm done for now! Come back next time you need help from the AI-powered Cooking Assistant!
